# Генерация текстов на основе LSTM

In [17]:
import re
import random
import nltk
import torch
import zipfile
import sqlite3
import numpy as np
import pandas as pd
from collections import Counter
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from nltk.corpus import wordnet

nltk.download('punkt')
nltk.download('wordnet')
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
!wget https://s3.amazonaws.com/text-datasets/nietzsche.txt

--2023-11-15 16:36:38--  https://s3.amazonaws.com/text-datasets/nietzsche.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.68.166, 52.217.230.208, 54.231.203.168, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.68.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 600901 (587K) [text/plain]
Saving to: ‘nietzsche.txt’

nietzsche.txt       100%[===================>] 586.82K  1.88MB/s    in 0.3s    

2023-11-15 16:36:39 (1.88 MB/s) - ‘nietzsche.txt’ saved [600901/600901]



In [3]:
with open('nietzsche.txt', 'r', encoding='utf-8') as file:
  text = file.read().lower()

длина всего корпуса:

In [ ]:
len(text)

600893

количество предложений:

In [ ]:
len(nltk.tokenize.sent_tokenize(text))

2864

всего уникальных символов:

In [ ]:
len(Counter(text))

57

Построим torch vocabulary из уникальных символов для обучения будущей модели. В данном случае целевой переменной будет следующий символ за последовательностью символов:

In [4]:
vocab = build_vocab_from_iterator([sorted(Counter(text))], min_freq=1,
                                  specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])
print(vocab.get_itos())

maxlen = 40
step = 3
X_train, Y_train = [], []
# получение последовательности в 40 символов каждые 3 символа
for i in range(0, len(text) - maxlen, step):
    # сохранение последовательности символов
    X_train.append(vocab(list(text[i: i + maxlen])))
    # сохранение первого символа после последовательности
    Y_train.append(vocab(list(text[i + maxlen])))

print("Всего последовательностей:", len(X_train))
# преобразование полученных чисел в тензоры
X_train = torch.tensor(X_train, dtype=torch.float32)
Y_train = torch.tensor(Y_train)
# дополнительное измерение для lstm слоя
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
Y_train = Y_train.reshape(Y_train.shape[0])
print("Размеры Тензоров:", X_train.shape, Y_train.shape)
vector_X = TensorDataset(X_train, Y_train)
train_loader = DataLoader(vector_X, batch_size=1024, shuffle=False)
for X, Y in train_loader:
    print("Размеры батчей:", X.shape, Y.shape)
    break

['<unk>', '\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ä', 'æ', 'é', 'ë']
Всего последовательностей: 200285
Размеры Тензоров: torch.Size([200285, 40, 1]) torch.Size([200285])
Размеры батчей: torch.Size([1024, 40, 1]) torch.Size([1024])


Итого у нас 200 тысяч данных для обучения с 40 признаками. Таргет - один из 58 символов. Vocab кодирует данные по принципу LabelEncoder, то ессть каждому символу присваивается соответствующее число.

Второй вариант обучения - предсказывать следующее слово:

In [11]:
# очистка слов от лишних символов (всё, что не латинская буква или пробел)
words = re.sub(r'[^a-z\s]', ' ', text)
words = re.sub(r'[\s]+', ' ', words).split()
# проверка, что слово существует (написано без опечатки)
words = list(filter(lambda word: len(wordnet.synsets(word)) > 0, words))
vocab_words = build_vocab_from_iterator([sorted(Counter(words))], min_freq=1,
                                  specials=["<unk>"])
vocab_words.set_default_index(vocab_words["<unk>"])
print("Слов в словаре:", len(vocab_words))

maxlen = 40
step = 3
X_train_words, Y_train_words = [], []
# получение последовательности в 40 слов через каждые 3 слова
for i in range(0, len(words) - maxlen, step):
    # сохранение последовательности слов
    X_train_words.append(vocab_words(words[i: i + maxlen]))
    # сохранение первого слова после последовательности
    Y_train_words.append(vocab_words[words[i + maxlen]])


print("Всего последовательностей:", len(X_train_words))
# преобразование полученных чисел в тензоры
X_train_words = torch.tensor(X_train_words, dtype=torch.float32)
Y_train_words = torch.tensor(Y_train_words)
# дополнительное измерение для lstm слоя
X_train_words = X_train_words.reshape(X_train_words.shape[0], X_train_words.shape[1], 1)
Y_train_words = Y_train_words.reshape(Y_train_words.shape[0])
print("Размеры Тензоров:", X_train_words.shape, Y_train_words.shape)
vector_X = TensorDataset(X_train_words, Y_train_words)
train_loader_words = DataLoader(vector_X, batch_size=1024, shuffle=False)
for X, Y in train_loader_words:
    print("Размеры батчей:", X.shape, Y.shape)
    break

Слов в словаре: 9138
Всего последовательностей: 23450
Размеры Тензоров: torch.Size([23450, 40, 1]) torch.Size([23450])
Размеры батчей: torch.Size([1024, 40, 1]) torch.Size([1024])


Данных для обучения почти в 10 раз меньше, а количество таргетов гораздо больше - 9 тысяч уникальных слов. Модель нужно подготовить сложнее, но скорость обучения будет больше.

## модель LSTM для генерации текста

In [5]:
class TextGeneratorLetters(torch.nn.Module):
    """Модель для генерации текста посимвольно"""
    def __init__(self):
        super().__init__()
        self.lstm = torch.nn.LSTM(input_size=1, hidden_size=256, num_layers=2,
                                  batch_first=True)
        self.dropout = torch.nn.Dropout(0.25)
        self.linear = torch.nn.Linear(256, len(vocab))

    def forward(self, X_batch):
        hidden = torch.randn(2, len(X_batch), 256).to(device)
        carry = torch.randn(2, len(X_batch), 256).to(device)
        output, _ = self.lstm(X_batch, (hidden, carry))
        output = self.dropout(output)
        output = self.linear(output[:, -1])
        return output

In [12]:
class TextGeneratorWords(torch.nn.Module):
    """Модель для генерации текста пословно"""
    def __init__(self):
        super().__init__()
        self.lstm = torch.nn.LSTM(input_size=1, hidden_size=1024, num_layers=2,
                                  batch_first=True)
        self.dropout = torch.nn.Dropout(0.25)
        self.linear = torch.nn.Linear(1024, 4096)
        self.dropout1 = torch.nn.Dropout(0.2)
        self.linear1 = torch.nn.Linear(4096, len(vocab_words))

    def forward(self, X_batch):
        hidden = torch.randn(2, len(X_batch), 1024).to(device)
        carry = torch.randn(2, len(X_batch), 1024).to(device)
        output, _ = self.lstm(X_batch, (hidden, carry))
        output = self.dropout(output)
        output = self.linear(output[:, -1])
        output = self.dropout1(output)
        output = self.linear1(output)
        return output

## Генерация текста

In [8]:
def generate(model, X_train, vocab, sep=""):
  # seed необходим для детерменированности результата
  random.seed(10)
  # выбираем случайную последовательность символов из датасета
  idx = random.randint(0, len(X_train))
  # генерируем новый символ после группы символов
  pattern = X_train[idx].numpy().astype(int).flatten().tolist()
  print("\033[92m Стартовый текст: {}".format(sep.join(vocab.lookup_tokens(pattern))))
  text = []
  for i in range(100):
    with torch.no_grad(): # не обновляя веса модели предсказываем значение
      X_batch = torch.tensor(pattern,
                             dtype=torch.float32).reshape(1, len(pattern), 1)
      preds = model(X_batch.to(device))
      # выбираем символ с наибольшей вероятностью
      symbol = preds.argmax(dim=-1).cpu().numpy()[0]
      text.append(symbol)
      pattern.append(symbol)
      pattern = pattern[1:]
  print("\033[91m Генерация: {}".format(sep.join(vocab.lookup_tokens(text))))

### Функция обучения

In [9]:
def train(model, train_loader, epochs, lr, X_train, vocab, sep=''):
  criterion = torch.nn.CrossEntropyLoss().to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=lr)
  for i in range(1, epochs+1):
    losses = []
    for X, Y in tqdm(train_loader):
      X, Y = X.to(device), Y.to(device)
      preds = model(X) # прогон признаков через слои модели
      loss = criterion(preds, Y) # вычисление функции ошибки
      losses.append(loss.item()) # сохранение вычисленной ошибки
      optimizer.zero_grad() # очистка вычисленных градиентов
      loss.backward() # вычисление градиента функции потерь по всем параметрам модели
      optimizer.step() # оптимизация весов модели
    print("Эпоха: {}".format(i), end=' | ')
    print("Ошибка : {:.3f}".format(torch.tensor(losses).mean()))
    # генерация по отному и тому же изначальному шаблону после каждой эпохи
    generate(model, X_train, vocab, sep)

### Обучение моделей:

In [10]:
model = TextGeneratorLetters().to(device)
print(model)
train(model, train_loader, 50, 1e-3, X_train, vocab)

TextGeneratorLetters(
  (lstm): LSTM(1, 256, num_layers=2, batch_first=True)
  (dropout): Dropout(p=0.25, inplace=False)
  (linear): Linear(in_features=256, out_features=58, bias=True)
)


100%|██████████| 196/196 [00:15<00:00, 12.58it/s]


Эпоха: 1 | Ошибка : 2.923
 Стартовый текст: nd true. here
is predicated of the effec
 Генерация:  the the the the the the the the the the the the the the the the the the the the the the the the the


100%|██████████| 196/196 [00:15<00:00, 12.84it/s]


Эпоха: 2 | Ошибка : 2.726
 Стартовый текст: nd true. here
is predicated of the effec
 Генерация:  the the the the the the the the the the the the the the the the the the the the the the the the the


...

Эпоха: 48 | Ошибка : 1.739
 Стартовый текст: nd true. here
is predicated of the effec
 Генерация: t of the seni of the sense of the strength of the strength of the streng of the prieit of the world 


100%|██████████| 196/196 [00:16<00:00, 11.96it/s]


Эпоха: 49 | Ошибка : 1.733
 Стартовый текст: nd true. here
is predicated of the effec
 Генерация: t of the streng and the self of the soirit of the strength of the self of the streng and the workd o


100%|██████████| 196/196 [00:16<00:00, 12.20it/s]


Эпоха: 50 | Ошибка : 1.725
 Стартовый текст: nd true. here
is predicated of the effec
 Генерация: tion of the streritation of the will of the streng of the sense of the senfer of the self of the sam


Первая модель, обученная на символах, обучилась до довольно низкого значения ошибки, она способна генерировать целые слова из языка, на котором обучалась, но имеет свойство зацикливаться и генерировать одинаковые последовательности символов. Это может быть связано с тем, что модель запомнила, какой символ идёт после одной и той же группы символов и всегда выдаёт этот символ, так как у него наибольшая вероятность среди всех остальных.

In [13]:
model = TextGeneratorWords().to(device)
print(model)
train(model, train_loader_words, 50, 1e-3, X_train_words, vocab_words, sep=' ')

TextGeneratorWords(
  (lstm): LSTM(1, 1024, num_layers=2, batch_first=True)
  (dropout): Dropout(p=0.25, inplace=False)
  (linear): Linear(in_features=1024, out_features=4096, bias=True)
  (dropout1): Dropout(p=0.2, inplace=False)
  (linear1): Linear(in_features=4096, out_features=9138, bias=True)
)


100%|██████████| 23/23 [00:24<00:00,  1.08s/it]


Эпоха: 1 | Ошибка : 7.926
 Стартовый текст: life blooms flowers only through kind feeling cheerfulness friendliness kindness a heart are unfailing sources impulse have made far more civilization other more noised manifestations it are styled sympathy benevolence sacrifice but it is customary depreciate little tokens kindly feeling
 Генерация: a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a he is a a a a a a a a a a a a a a a a a a a a a a a a a he a a a a a a a a a a a a a a a a a a he a a a a a a a a a a a a a a a a a a a a


100%|██████████| 23/23 [00:24<00:00,  1.05s/it]


Эпоха: 2 | Ошибка : 7.356
 Стартовый текст: life blooms flowers only through kind feeling cheerfulness friendliness kindness a heart are unfailing sources impulse have made far more civilization other more noised manifestations it are styled sympathy benevolence sacrifice but it is customary depreciate little tokens kindly feeling
 Генерация: in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in in


...

Эпоха: 48 | Ошибка : 6.438
 Стартовый текст: life blooms flowers only through kind feeling cheerfulness friendliness kindness a heart are unfailing sources impulse have made far more civilization other more noised manifestations it are styled sympathy benevolence sacrifice but it is customary depreciate little tokens kindly feeling
 Генерация: in in in is in is in in in is in in in is in in in in is in in in in in in in is in is in is in in in in in is in in in in in in is in in in in in in in in is is in in is in in in in so in in in in in is in in in in is in in so in in is it in is in in in in in in in in in in in in is in is in in in


100%|██████████| 23/23 [00:23<00:00,  1.04s/it]


Эпоха: 49 | Ошибка : 6.411
 Стартовый текст: life blooms flowers only through kind feeling cheerfulness friendliness kindness a heart are unfailing sources impulse have made far more civilization other more noised manifestations it are styled sympathy benevolence sacrifice but it is customary depreciate little tokens kindly feeling
 Генерация: is is is is is is is is is is is in is is is is is is in in in is is is is is is is in is is in is is is in is is is is is in is in is is is is is is is in is in is is is in is in in is in is is is is is in is is is in in is is in is is is is is in is is is is is is is is is in is is in in is is in


100%|██████████| 23/23 [00:23<00:00,  1.04s/it]


Эпоха: 50 | Ошибка : 6.385
 Стартовый текст: life blooms flowers only through kind feeling cheerfulness friendliness kindness a heart are unfailing sources impulse have made far more civilization other more noised manifestations it are styled sympathy benevolence sacrifice but it is customary depreciate little tokens kindly feeling
 Генерация: in is is is is is is is in is in in in in in in is in is in in in in is in in is in is in in in is is in is in in in in in is in is in in in is in in in is in is in is in in in is in in is in in in in is in in in is in is is in is in is in in in is in in is in in in is in in in in in is is is is is


Модель, обученная на словах имеет точность сильно ниже из-за меньшего размера датасета и большего размера целевой переменной. В основном выдаёт последовательности из наиболее частных слов, встречаемых в датасете.

## Генерация текста для РУССКОЯЗЫЧНОГО НАБОРА глав Wikibooks.

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
zf = zipfile.ZipFile('/content/drive/MyDrive/archive.zip')
for file in tqdm(zf.infolist()):
    zf.extract(file)

100%|██████████| 1/1 [01:55<00:00, 115.04s/it]


In [35]:
con = sqlite3.connect('/content/wikibooks.sqlite')
cur = con.cursor()
try:
  cur.execute("""SELECT * FROM ru""")
  fa = cur.fetchall()
  columns = [i[0] for i in cur.description]
  data = pd.DataFrame(fa, columns=columns)
  display(data.head())
except sqlite3.DatabaseError as error:
  print("Ошибка:", error)
finally:
  cur.close()
  con.close()
text = data['body_text'].str.cat(sep=' ').lower()

,title,url,abstract,body_text,body_html
0,Викиучебник: Техника и технология средств масс...,https://ru.wikibooks.org/wiki/%D0%A2%D0%B5%D1%...,* [станция|Рабочая станция];,Рабочая станция;\nСервер;\nПерсональный компью...,"<div class=""mw-parser-output""><ul><li><a href=..."
1,Викиучебник: АОН/Пилотское свидетельство,https://ru.wikibooks.org/wiki/%D0%90%D0%9E%D0%...,Гражданское пилотское свидетельство - разрешен...,В Википедии имеется статья по теме «Свидетельс...,"<div class=""mw-parser-output""><div class=""info..."
2,Викиучебник: Книга программиста/Структуры данн...,https://ru.wikibooks.org/wiki/%D0%9A%D0%BD%D0%...,К оглавлению,"К оглавлению\nВсе программы, код которых вылож...","<div class=""mw-parser-output""><p><a href=""/wik..."
3,Викиучебник: Тесты НМО/Гигиенические основы и ...,https://ru.wikibooks.org/wiki/%D0%A2%D0%B5%D1%...,Гигиенические основы и медицинский контроль за...,Гигиенические основы и медицинский контроль за...,"<div class=""mw-parser-output""><p><b>Гигиеничес..."
4,Викиучебник: Коктейли/Пенная фея,https://ru.wikibooks.org/wiki/%D0%9A%D0%BE%D0%...,Пенная фея,Пенная фея\n\nДжин Old Tom — 60 г\nАбсент — 15...,"<div class=""mw-parser-output""><p><b>Пенная фея..."


In [37]:
# очистка текста от всего, кроме кириллицы и пунктуации
text = re.sub(r'[^а-яё\s,.!?()\"\":;-]', ' ', text)
text = re.sub(r'[\s]+', ' ', text)
print("Длина текста:", len(text))
vocab = build_vocab_from_iterator([sorted(Counter(text))], min_freq=1,
                                  specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])
print(vocab.get_itos())

maxlen = 30
step = 10
X_train, Y_train = [], []
# получение последовательности в 30 символов каждые 10 символа
for i in range(0, len(text) - maxlen, step):
    # сохранение последовательности символов
    X_train.append(vocab(list(text[i: i + maxlen])))
    # сохранение первого символа после последовательности
    Y_train.append(vocab(list(text[i + maxlen])))

print("Всего последовательностей:", len(X_train))
# преобразование полученных чисел в тензоры
X_train = torch.tensor(X_train, dtype=torch.float32)
Y_train = torch.tensor(Y_train)
# дополнительное измерение для lstm слоя
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
Y_train = Y_train.reshape(Y_train.shape[0])
print("Размеры Тензоров:", X_train.shape, Y_train.shape)
vector_X = TensorDataset(X_train, Y_train)
train_loader = DataLoader(vector_X, batch_size=1024, shuffle=False)
for X, Y in train_loader:
    print("Размеры батчей:", X.shape, Y.shape)
    break

Длина текста: 41907187
['<unk>', ' ', '!', '"', '(', ')', ',', '-', '.', ':', ';', '?', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я', 'ё']
Всего последовательностей: 4190716
Размеры Тензоров: torch.Size([4190716, 30, 1]) torch.Size([4190716])
Размеры батчей: torch.Size([1024, 30, 1]) torch.Size([1024])


Будем обучать модель на 4 миллионах очищенных данных. Целевая переменная - буква кириллицы или знак пунктуации.

In [38]:
model = TextGeneratorLetters().to(device)
print(model)
train(model, train_loader, 10, 1e-3, X_train, vocab)

TextGeneratorLetters(
  (lstm): LSTM(1, 256, num_layers=2, batch_first=True)
  (dropout): Dropout(p=0.25, inplace=False)
  (linear): Linear(in_features=256, out_features=45, bias=True)
)


100%|██████████| 4093/4093 [04:18<00:00, 15.83it/s]


Эпоха: 1 | Ошибка : 2.502
 Стартовый текст: асштабам проект. никому в то в
 Генерация:  оаразение процессвоение процессвенной польтой проводни и просоятия и постедние представления переме


100%|██████████| 4093/4093 [04:19<00:00, 15.77it/s]


Эпоха: 2 | Ошибка : 2.061
 Стартовый текст: асштабам проект. никому в то в
 Генерация: ыбора постояние прододство процесси при продолжением и продрамма просода проценски в образом и проце


...

Эпоха: 8 | Ошибка : 1.671
 Стартовый текст: асштабам проект. никому в то в
 Генерация: елосоддо примечания противогазных степени программиров в составление по серверам по образом прогоамм


100%|██████████| 4093/4093 [04:18<00:00, 15.82it/s]


Эпоха: 9 | Ошибка : 1.649
 Стартовый текст: асштабам проект. никому в то в
 Генерация: ессия при помощи производственной серверах и получения программирования процесса проверки программы 


100%|██████████| 4093/4093 [04:18<00:00, 15.85it/s]


Эпоха: 10 | Ошибка : 1.631
 Стартовый текст: асштабам проект. никому в то в
 Генерация: елтора продажа проверяются в своих строки происходит подавать полазывающийся провериальные сервера п


In [39]:
train(model, train_loader, 10, 3e-4, X_train, vocab)

100%|██████████| 4093/4093 [04:18<00:00, 15.84it/s]


Эпоха: 1 | Ошибка : 1.592
 Стартовый текст: асштабам проект. никому в то в
 Генерация: ертии состояния по семуерани и процесс против продуктов править выберите один ответ: . . . . . . . .


100%|██████████| 4093/4093 [04:18<00:00, 15.85it/s]


Эпоха: 2 | Ошибка : 1.581
 Стартовый текст: асштабам проект. никому в то в
 Генерация: озможность простой строки править выберите один ответ: . . . . . . . . . . . . . . . . . . . . . . .


100%|██████████| 4093/4093 [04:18<00:00, 15.86it/s]


...

Эпоха: 8 | Ошибка : 1.548
 Стартовый текст: асштабам проект. никому в то в
 Генерация: се от представления при помощью составляет порледовательность предположительно по строми править выб


100%|██████████| 4093/4093 [04:18<00:00, 15.84it/s]


Эпоха: 9 | Ошибка : 1.544
 Стартовый текст: асштабам проект. никому в то в
 Генерация: ерхней человек производственных систем и производства и проведения проверка просто процедуры простра


100%|██████████| 4093/4093 [04:17<00:00, 15.88it/s]


Эпоха: 10 | Ошибка : 1.540
 Стартовый текст: асштабам проект. никому в то в
 Генерация: ремя положения при приводит к просто образуются составляющий клетки при проведении простого процессо


Модель обучилась до достаточно низкого значения ошибки. Она генерирует осмысленные слова, но не последовательности слов.